<a href="https://colab.research.google.com/github/AmSharma05/NaiveBayesvsNN/blob/main/SpamNNfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Keras-Preprocessing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import re
import string
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional
import tensorflow as tf
from matplotlib import pyplot as plt

# Load data
data = pd.read_csv("spam.csv", encoding="'latin'")
data["text"] = data.v2
data["spam"] = data.v1

# Split data into training and testing sets
emails_train, emails_test, target_train, target_test = train_test_split(data.text, data.spam, test_size=0.2)

# Define text cleaning functions
def remove_hyperlink(word):
    return re.sub(r"http\S+", "", word)

def to_lower(word):
    return word.lower()

def remove_number(word):
    return re.sub(r'\d+', '', word)

def remove_punctuation(word):
    return word.translate(str.maketrans(dict.fromkeys(string.punctuation)))

def remove_whitespace(word):
    return word.strip()

def replace_newline(word):
    return word.replace('\n', '')

def clean_up_pipeline(sentence):
    cleaning_utils = [remove_hyperlink, replace_newline, to_lower, remove_number, remove_punctuation, remove_whitespace]
    for o in cleaning_utils:
        sentence = o(sentence)
    return sentence

# Clean up training and testing data
x_train = [clean_up_pipeline(o) for o in emails_train]
x_test = [clean_up_pipeline(o) for o in emails_test]

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y = le.fit_transform(target_train.values)
test_y = le.transform(target_test.values)

# Define model parameters
embed_size = 100 # how big is each word vector
max_feature = 50000 # how many unique words to use (i.e num rows in embedding vector)
max_len = 2000 # max number of words in a question to use

# Tokenize text data
tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(x_train)

x_train_features = np.array(tokenizer.texts_to_sequences(x_train))
x_test_features = np.array(tokenizer.texts_to_sequences(x_test))

# Pad sequences
x_train_features = pad_sequences(x_train_features, maxlen=max_len)
x_test_features = pad_sequences(x_test_features, maxlen=max_len)

# Build model
embedding_vecor_length = 32
model = tf.keras.Sequential()
model.add(Embedding(max_feature, embedding_vecor_length, input_length=max_len))
model.add(Bidirectional(tf.keras.layers.LSTM(64)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(x_train_features, train_y, batch_size=512, epochs=20, validation_data=(x_test_features, test_y))

# Evaluate model performance
pred_y = model.predict(x_test_features)
pred_y = np.round(pred_y.flatten())

precision = precision_score(test_y, pred_y)
recall = recall_score(test_y, pred_y)
f1 = f1_score(test_y, pred_y)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


<ipython-input-7-961ba4965caa>:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train_features = np.array(tokenizer.texts_to_sequences(x_train))
<ipython-input-7-961ba4965caa>:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test_features = np.array(tokenizer.texts_to_sequences(x_test))


Epoch 1/20
9/9 [==============================] - 161s 17s/step - loss: 0.6533 - accuracy: 0.8281 - val_loss: 0.5756 - val_accuracy: 0.8592
Epoch 2/20
9/9 [==============================] - 163s 18s/step - loss: 0.4618 - accuracy: 0.8676 - val_loss: 0.4143 - val_accuracy: 0.8592
Epoch 3/20
9/9 [==============================] - 161s 18s/step - loss: 0.3662 - accuracy: 0.8676 - val_loss: 0.3650 - val_accuracy: 0.8592
Epoch 4/20
9/9 [==============================] - 163s 18s/step - loss: 0.3271 - accuracy: 0.8676 - val_loss: 0.2999 - val_accuracy: 0.8592
Epoch 5/20
9/9 [==============================] - 172s 19s/step - loss: 0.2309 - accuracy: 0.8840 - val_loss: 0.1901 - val_accuracy: 0.9426
Epoch 6/20
9/9 [==============================] - 160s 18s/step - loss: 0.1472 - accuracy: 0.9690 - val_loss: 0.1498 - val_accuracy: 0.9641
Epoch 7/20
9/9 [==============================] - 161s 18s/step - loss: 0.1006 - accuracy: 0.9823 - val_loss: 0.1016 - val_accuracy: 0.9731
Epoch 8/20
9/9 [====